In [1]:
!pip install minsearch


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import minsearch
import json 

In [3]:
with open('documents.json', 'rt') as  f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["questions", 'text', 'section'],
    keyword_fields=['course']
)

In [6]:
# SELECT * WHERE course = 'data-engineering-zoomcamp'

In [7]:
index.fit(documents)

In [8]:
import os
from openai import OpenAI
from dotenv import load_dotenv

base_url=os.environ.get("BASE_URL")
api_key=os.environ.get("API_KEY")

client = OpenAI(
    base_url=base_url,
    api_key=api_key,
)

In [9]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}


    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [10]:
def build_prompt(query, search_results):
    prompt_template = """
    Your are a course teaching assistant. 
    Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT  when answering the QUESTION. 
    If the CONTEXT doesn't containt the answer output None.

    QUESTION:
    {question}

    CONTEXT:
    {context}
    """.strip()
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt 


In [11]:
def llm(prompt):
    responce = client.chat.completions.create(
        model="deepseek/deepseek-chat-v3-0324:free",
        messages=[{
            "role":"user",
            "content":prompt
        }]
    )
    return responce.choices[0].message.content

In [12]:
query = "how do I run kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query=query, search_results=search_results)
    return llm(prompt=prompt)

In [13]:
rag(query)

"To run Kafka, follow these steps based on the provided context:\n\n1. **Start Kafka Broker**: Ensure the Kafka broker Docker container is running. Use `docker ps` to check its status. If it's not running, navigate to the folder containing your `docker-compose.yaml` file and run `docker compose up -d` to start all instances.\n\n2. **Install Required Dependencies**:\n   - For Python, install the necessary Kafka libraries:\n     - `pip install confluent-kafka` or `conda install conda-forge::python-confluent-kafka`\n     - If you encounter an error with `avro`, run `pip install confluent-kafka[avro]` or `pip install fastavro`.\n   - If you face issues with `kafka-python`, uninstall and reinstall a specific version:  \n     ```sh\n     pip uninstall kafka-python\n     pip install kafka-python==1.4.6\n     ```\n   - For Java, ensure dependencies are bundled in the JAR by modifying `build.gradle` (as shown in the context) and running `gradle shadowjar`.\n\n3. **Alternative to Kafka**: If you